In [3]:
#import packages
import pandas as pd
import numpy as np
from datetime import datetime as dt
today = dt.now()

#Read the downloaded report
Path_A = "/Users/swaroopbasavarajmusti/Desktop/G10andGetirBuyuk_AllProducts_without_TRDE.xlsx"

Assortment = pd.read_excel(Path_A)

#change the marketplace product id column name (not required but to avoid confusion)

Assortment.rename(columns={'marketproduct_id':'Getir Product Id'}, inplace=True)

#select only required columns

Assortment = Assortment[['createdatl','updatedatl','Getir Product Id','product_barcode','marketproduct_name_de',\
'segment_name','brand_name','category_name_en','subcategory_name_en','supplier_names',\
'net_net_buying_price_with_vat','list_price','g10_price','g10_struck_price','transfer_groups','isvisible']]

#filter unwanted segments from the dataframe
Segment_filters = ["Asset", "None", "Test"]
Assortment = Assortment[~Assortment.segment_name.isin(Segment_filters)].reset_index(drop = True)

#sort the values based on the date created
Assortment = Assortment.sort_values(by="createdatl",ascending = False).reset_index(drop = True)

#------------------------------

#creating dictionary to match the product ids

Path_B = "/Users/swaroopbasavarajmusti/Desktop/Inventory.xls"
Dictionary = pd.read_excel(Path_B)

Dictionary = Dictionary[Dictionary['Getir Product Id'].notna()].reset_index(drop = True)
Dictionary = Dictionary[['Getir Product Id','Product Id','Product SKU','Enabled / Disabled']]

#------------------------------

#left joining Panel assortment with eddress ID dictionary

Assortment_Inventory = pd.merge(Assortment, 
                      Dictionary, 
                      on ='Getir Product Id', 
                      how ='left')

Update_list = Assortment_Inventory.loc[Assortment_Inventory['segment_name'] == 'Delist'].reset_index(drop = True)
Update_list = Update_list.loc[Update_list['Product Id'].notna()].reset_index(drop = True)
Update_list = Update_list.loc[Update_list['Enabled / Disabled'] == 'Enabled'].reset_index(drop = True)
Update_list = Update_list[['Getir Product Id','Product Id','Product SKU',]]
Update_list.to_excel("/Users/swaroopbasavarajmusti/Desktop/Eddress_Status_Update_list.xlsx")